# Face Recognition Component

In [5]:
import sys
!{sys.executable} -m pip install opencv-python

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 MB 2.0 MB/s eta 0:00:0000:0100:01
  DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python3.9 -m pip install --upgrade pip


In [6]:
import cv2
import time
import numpy as np

# Preprocess the Data

First, we have to standardize the data for use. First, we process it to ensure that the height and width is a standard size. In our case, we will use the standard resolution of 1280x720 and aspect ratio of 16:9. Furthermore, the video is preprocessed in grayscale to help with classification.

In [ ]:
# Create our face classifier
# can change path based on where you loaded the haarcascade_classifier
face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# Initiate video capture for video file
cap = cv2.VideoCapture('sample.avi')
fps = 25

# Make sure the path is pointing correctly, else print error
if cap.isOpened()== False:
    print("Error opening the video file. Check your file path for typos. Or move the movie file to the same location as this script/notebook")

# Loop once video is successfully loaded
while cap.isOpened():

    time.sleep(.05)

    # Read first frame
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Pass frame to our face classifier
    faces = face_classifier.detectMultiScale(gray, 1.05, 16)

    # Extract bounding boxes for any faces identified
    for (x,y,w,h) in faces:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 255), 2)
        cv2.imshow('Faces', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'): #13 is the Enter Key
        break

cap.release()
cv2.destroyAllWindows()

As we can see when running this code, we ran the detectMultiScale method with parameters: imageScale = 2 and minNeighbors = 8. imageScale essentially changes the size of the face if detected in the image if present; by rescaling the input image, it allows the algorithm to map the faces in the input image to the images it was trained on, raising its accuracy. If this number is high, say 4, it skips multiple levels of image "zooms", increasing speed with less calculations but also potentially decreasing accuracy as faces of a certain size may no longer be detected by an algorithm that runs a smaller imageScale. minNeighbors directly correlates to the quality of the face, the "sureness" of the algorithm. When set too low, the algorithm may detect faces that are not actually there. Increasing this number raises the amount of conditions the "face" that the classifier is considering has to fulfill to be returned as a face. So by increasing this number, you add more "sureness" to the algorithm for it to return that it is a face.

Now, we need to ensure that the file is saved appropriately with the bounding boxes. Or find a way to save just the bounding boxes.

In [9]:
# Create our face classifier
# can change path based on where you loaded the haarcascade_classifier
face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# Initiate video capture for video file
cap = cv2.VideoCapture('sample.mp4')
fps = 25

# Automatically grab width and height from video feed
# (returns float which we need to convert to integer for later on!)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# IMPORT IN THE VIDEO WRITER METHOD
# MACOS AND LINUX: *'XVID' (MacOS users may want to try VIDX as well just in case)
# WINDOWS *'VIDX'
writer = cv2.VideoWriter('sample_faces.mp4', cv2.VideoWriter_fourcc(*'XVID'),25, (width, height))

# Make sure the path is pointing correctly, else print error
if cap.isOpened()== False:
    print("Error opening the video file. Check your file path for typos. Or move the movie file to the same location as this script/notebook")

# Set up the label parameters
label = 'FACE'
font = cv2.FONT_HERSHEY_SIMPLEX
font_scale = 0.5
font_thickness = 2

# Loop once video is successfully loaded
while cap.isOpened():

    time.sleep(.05)

    # Read first frame
    ret, frame = cap.read()

    # break out of loop if we have finished reading through the video
    if frame is None:
        break

    # get the frame in grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Pass frame to our face classifier
    faces = face_classifier.detectMultiScale(gray, 1.08, 23)

    # Extract bounding boxes for any faces identified
    for (x,y,w,h) in faces:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 255), 2)
        # no need to print out the faces anymore, as they are saved when capturing the frames
        #cv2.imshow('Faces', frame)
        
        label_size = cv2.getTextSize(label, font, font_scale, font_thickness)[0]
        label_top_left = (x,y+14)
        label_bottom_right = (x + label_size[0], y)
        cv2.putText(frame, label, (x, y), font, font_scale, (0, 255, 255), font_thickness)


    writer.write(frame)

cap.release()
writer.release()
cv2.destroyAllWindows()